In [1]:
import numpy as np
import pandas as pd
import pyodbc
import re

In [76]:
SERVER = '(local)'
DATABASE = 'Checks'
USERNAME = 'VORONVI\Vladi'

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};Trusted_Connection=Yes;TrustServerCertificate=Yes'
conn = pyodbc.connect(connectionString)

SQL_QUERY = """
SELECT 
Id,
Name,
YandexGPT,
Category
FROM 
ItemData;
"""

cursor = conn.cursor()
cursor.execute(SQL_QUERY)

records = cursor.fetchall()
    
df = pd.DataFrame.from_records(records, columns=['Id', 'Name', 'YandexGPT', 'Category'])
df

,Id,Name,YandexGPT,Category
0,24CBA3DB-EEA3-44A5-BD6D-017F7FA86CE1,*GL.VIL.Кукуруза сахар.ж/б 425мл,Кукуруза,продукт
1,3C89D571-C6B8-4BA2-9A06-01F25E50D680,ЧУДО Йогурт КЛУБ-КИВИ фрук.260г,Йогурт,продукт
2,AC339F91-8DBC-4EB1-BB50-026850EC67BD,Яйцо куриное Деревенское С1 10шт бокс:20,Яйца,продукт
3,5BFF7D6F-607C-41DA-8A57-03656AEFFFA7,МАГНИТ Крупа гречневая ядрица в/с 800-900г п/,Крупа,продукт
4,D48C17F5-9987-44B2-BBAC-03EACF2F8B74,*Игр.ТАЙНА ПЯТОЙ ПЛАНЕТЫ K012024,Игра,Неизвестно
...,...,...,...,...
1030,5C91EED1-0C38-4CCA-8E63-FC23098CFD05,*Дрожжи САФ-МОМЕНТ хл/п сух.11г,Дрожжи,ингредиент
1031,731B5679-B057-4108-B26A-FD8234BEB2A3,БАНАНЫ 1кг,Бананы,фрукты
1032,B75B726E-5A2E-41F5-9105-FD8B2C8B3DD9,ПЯТЕРОЧКА Пакет майка зел.30х57,Пакет,прочее
1033,94E1F7B2-0447-4AB3-9002-FE4593295D58,Термоэтикетки д/яиц 7 шт KE160702 МЮ (СИ):60/,Термоэтикетки,товар


In [73]:
print(df.YandexGPT.tolist())

['Анализ', 'Апельсины', 'Артефакт', 'БАД', 'Бальзам', 'Банан', 'Бананы', 'Батончик', 'Билет', 'Билеты', 'Блюдо', 'Бритва', 'Бумага', 'Вафли', 'Вафля', 'Ветчина', 'Виноград', 'Вода', 'Выпечка', 'Гель', 'Геркулес', 'Гречка', 'Грибы', 'Губки', 'Десерт', 'Детриойл', 'Донат', 'Доставка', 'Дрожжи', 'Жевательная резинка', 'Жилье', 'Закладка', 'Зелень', 'Зефир', 'Зубная паста', 'Игра', 'Игрушка', 'Ирис', 'Йогурт', 'Какао', 'Кеды', 'Кетчуп', 'Кефир', 'Киви', 'Килька', 'Клатч', 'Клей', 'Книга', 'Колготки', 'Консервы', 'Контент', 'Конфеты', 'Конфитюр', 'Концентрат', 'Копия', 'Корж', 'Коржи', 'Корм', 'Кофе', 'Краб', 'Краситель', 'Круассан', 'Крупа', 'Кукуруза', 'Кукурузные палочки', 'Кулич', 'Лак', 'Лапша', 'Леденец', 'Лекарство', 'Лимоны', 'Лобио', 'Майонез', 'Макароны', 'Мандарины', 'Маска', 'Масло', 'Мешки', 'Молоко', 'Морковь', 'Морковь по-корейски', 'Мороженое', 'Морс', 'Мука', 'Мыло', 'Мюсли', 'Мясное изделие', 'Мясной продукт', 'Мясо', 'Мясопродукты', 'Наклейка-патч', 'Напиток', 'Насадка', 

In [4]:
def levenstein(str_1, str_2):
    n, m = len(str_1), len(str_2)
    if n > m:
        str_1, str_2 = str_2, str_1
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if str_1[j - 1] != str_2[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [5]:
print(levenstein("Молоко", "молоко"))
mystring = "Ч-ХЛЕБ Рулет с маковой нач.200г ЧЕЛНЫ-ХЛ"
print(mystring.replace('.',' ').replace('-',' ').split())

1
['Ч', 'ХЛЕБ', 'Рулет', 'с', 'маковой', 'нач', '200г', 'ЧЕЛНЫ', 'ХЛ']


In [6]:
class Product:
    def __init__(self, name):
        self.arrayNames = [name.upper(), name.lower(), name.capitalize()]
        

In [7]:
products = [Product("услуга"), Product("молоко"), Product("кукуруза"), Product("фасоль"), Product("бананы"), Product("пакет"), Product("кофе"), Product("консервы"), Product("семена"), Product("хлеб"), Product("шоколад"), Product("творог"), Product("сахар"), Product("бумага"), Product("печенье"),
             Product("десерт"), Product("кетчуп"), Product("сгущенка"), Product("кино"), Product("билет"), Product("подписка"), Product("вода"), Product("рис"), Product("яйцо"), Product("морковь"), Product("мандарины"), Product("йогурт"), Product("макароны"), Product("мюсли"), Product("батончик"),
             Product("огурцы"), Product("яблоки"), Product("сок"), Product("мыло"), Product("чай"), Product("томаты"), Product("сыр"), Product("жевачка"), Product("ряженка"), Product("пирог"), Product("цикорий"), Product("крупа"), Product("колбаса"), Product("сельдь"), Product("капуста"), Product("салат"),
             Product("лимон"), Product("масло"), Product("пряники"), Product("апельсины"), Product("сухарики"), Product("книга"), Product("клей"), Product("маска"), Product("чипсы"), Product("конфеты"), Product("ксерокопия"), Product("майонез"), Product("мука"), Product("шампунь"), Product("виноград"), Product("сосиски")]

In [34]:
print(len(products))
print(products[32].arrayNames[1])


62
сок


In [205]:
centroidsProduct = []
for product in products:
    arr = []
    for prod in products:
        tmplev = levenstein(product.arrayNames[1], prod.arrayNames[1],)
        arr.append(tmplev)
    centroidsProduct.append(arr)

In [206]:
print(centroidsProduct)

[[0, 5, 5, 6, 6, 6, 6, 7, 5, 5, 6, 6, 5, 4, 7, 6, 6, 6, 6, 5, 7, 5, 6, 6, 7, 9, 5, 8, 5, 8, 6, 5, 5, 5, 6, 6, 5, 6, 6, 5, 7, 4, 5, 5, 5, 5, 6, 5, 7, 8, 7, 4, 5, 5, 6, 7, 9, 7, 4, 6, 6, 6], [5, 0, 8, 5, 6, 6, 5, 7, 6, 5, 5, 5, 6, 6, 7, 6, 6, 7, 5, 5, 6, 5, 6, 5, 4, 8, 5, 6, 5, 7, 6, 3, 4, 3, 6, 5, 6, 6, 6, 5, 6, 6, 5, 5, 7, 5, 5, 4, 6, 8, 7, 6, 5, 4, 6, 6, 7, 5, 4, 7, 7, 5], [5, 8, 0, 8, 8, 7, 7, 7, 7, 8, 7, 7, 7, 6, 8, 7, 6, 7, 7, 8, 7, 7, 7, 8, 8, 8, 6, 6, 8, 8, 6, 8, 7, 8, 8, 8, 7, 7, 7, 7, 6, 4, 6, 8, 5, 8, 8, 8, 8, 9, 6, 6, 7, 7, 8, 7, 9, 7, 5, 7, 8, 8], [6, 5, 8, 0, 5, 5, 5, 7, 6, 6, 5, 6, 5, 6, 6, 5, 6, 8, 5, 6, 8, 5, 5, 5, 5, 8, 6, 6, 4, 6, 6, 5, 4, 5, 5, 6, 5, 7, 7, 5, 6, 6, 7, 5, 6, 5, 5, 3, 7, 7, 7, 6, 6, 4, 6, 7, 9, 5, 6, 5, 7, 6], [6, 6, 8, 5, 0, 5, 6, 6, 5, 6, 7, 6, 4, 4, 6, 6, 6, 7, 5, 5, 8, 5, 6, 6, 7, 4, 6, 4, 6, 5, 5, 6, 6, 6, 5, 4, 6, 6, 6, 6, 7, 6, 6, 6, 6, 4, 5, 5, 6, 7, 7, 5, 6, 5, 5, 5, 10, 5, 5, 5, 7, 7], [6, 6, 7, 5, 5, 0, 4, 7, 5, 4, 6, 6, 4, 6, 5, 4, 5, 7, 5, 

In [227]:
data = []
i=0
#products = ["услуга","молоко","кукуруза","фасоль","бананы","пакет","кофе","консервы","семена","хлеб","шоколад","творог","сахар","бумага","печенье","десерт","кетчуп","сгущенка","кино","билет","подписка","вода","рис", "яйцо", "морковь", "мандарины", "йогурт", "макароны"]
for nameProduct in df.Name:
    newarr = []
    c = 0
    for product in products:
        maxlev = 100
        for textInProduct in nameProduct.replace('.',' ').replace('-',' ').split():
            for prod in product.arrayNames:
                tmplev = levenstein(prod, textInProduct)
                if(tmplev < maxlev):
                    maxlev = tmplev
                if(maxlev == 0):
                    print(prod)
                    print(i)
                    c = 1
        newarr.append(maxlev)
    if(c == 1):
        print(newarr)
    i = i + 1
    data.append(newarr)
    #df[product.arrayNames[1]] = newarr
    #data.append(newarr)
print(data)

Подписка
0
ПОДПИСКА
0
подписка
0
Подписка
0
ПОДПИСКА
0
подписка
0
Подписка
0
[5, 6, 7, 5, 4, 4, 4, 7, 4, 4, 6, 6, 4, 5, 6, 6, 6, 6, 3, 5, 0, 3, 3, 4, 7, 7, 6, 7, 5, 6, 6, 6, 3, 4, 2, 5, 3, 6, 5, 5, 6, 4, 5, 6, 5, 4, 5, 4, 5, 8, 7, 3, 4, 4, 5, 6, 7, 6, 3, 6, 6, 4]
Подписка
1
ПОДПИСКА
1
подписка
1
Подписка
1
ПОДПИСКА
1
подписка
1
Подписка
1
[5, 6, 7, 5, 4, 4, 4, 7, 4, 4, 6, 6, 4, 5, 6, 6, 6, 6, 3, 5, 0, 3, 3, 4, 7, 7, 6, 7, 5, 6, 6, 6, 3, 4, 2, 5, 3, 6, 5, 5, 6, 4, 5, 6, 5, 4, 5, 4, 5, 8, 7, 3, 4, 4, 5, 6, 7, 6, 3, 6, 6, 4]
Подписка
2
ПОДПИСКА
2
подписка
2
Подписка
2
ПОДПИСКА
2
подписка
2
Подписка
2
[5, 6, 7, 5, 4, 4, 4, 7, 4, 4, 6, 6, 4, 5, 6, 6, 6, 6, 3, 5, 0, 3, 3, 4, 7, 7, 6, 7, 5, 6, 6, 6, 3, 4, 2, 5, 3, 6, 5, 5, 6, 4, 5, 6, 5, 4, 5, 4, 5, 8, 7, 3, 4, 4, 5, 6, 7, 6, 3, 6, 6, 4]
Подписка
3
ПОДПИСКА
3
подписка
3
Подписка
3
ПОДПИСКА
3
подписка
3
Подписка
3
[5, 6, 7, 5, 4, 4, 4, 7, 4, 4, 6, 6, 4, 5, 6, 6, 6, 6, 3, 5, 0, 3, 3, 4, 7, 7, 6, 7, 5, 6, 6, 6, 3, 4, 2, 5, 3, 6, 5, 5, 6, 4, 5, 6

Пакет
100
ПАКЕТ
100
пакет
100
Пакет
100
ПАКЕТ
100
пакет
100
Пакет
100
ПАКЕТ
100
пакет
100
Пакет
100
ПАКЕТ
100
пакет
100
Пакет
100
[5, 5, 7, 5, 5, 0, 4, 5, 4, 4, 5, 6, 4, 6, 5, 4, 5, 7, 5, 3, 5, 5, 3, 5, 6, 8, 5, 6, 5, 7, 6, 6, 4, 5, 4, 5, 4, 5, 5, 4, 6, 5, 5, 4, 5, 3, 5, 4, 6, 7, 6, 5, 4, 4, 5, 5, 6, 5, 4, 6, 7, 5]
Пакет
102
ПАКЕТ
102
пакет
102
Пакет
102
ПАКЕТ
102
пакет
102
Пакет
102
ПАКЕТ
102
пакет
102
Пакет
102
ПАКЕТ
102
пакет
102
Пакет
102
[5, 4, 7, 5, 5, 0, 4, 7, 5, 3, 6, 6, 4, 4, 5, 4, 5, 6, 4, 3, 6, 4, 3, 4, 5, 6, 5, 5, 4, 6, 6, 5, 3, 3, 3, 5, 3, 4, 5, 4, 6, 4, 6, 4, 5, 3, 5, 3, 6, 7, 6, 4, 3, 1, 5, 5, 7, 4, 2, 6, 7, 6]
Цикорий
103
ЦИКОРИЙ
103
цикорий
103
Цикорий
103
ЦИКОРИЙ
103
цикорий
103
Цикорий
103
ЦИКОРИЙ
103
цикорий
103
Цикорий
103
ЦИКОРИЙ
103
цикорий
103
Цикорий
103
[5, 5, 6, 4, 5, 3, 4, 7, 6, 4, 5, 5, 4, 5, 7, 4, 6, 7, 4, 4, 7, 4, 2, 4, 6, 7, 5, 6, 4, 6, 6, 5, 3, 4, 3, 5, 3, 6, 6, 4, 0, 4, 5, 6, 4, 3, 5, 3, 6, 8, 6, 4, 4, 3, 4, 6, 7, 6, 4, 6, 5, 6]
Семена
104
СЕМЕНА
104
с

[5, 4, 7, 4, 5, 5, 4, 7, 0, 3, 6, 6, 5, 4, 4, 4, 5, 4, 4, 4, 6, 4, 3, 4, 7, 8, 6, 7, 4, 5, 6, 4, 3, 3, 3, 5, 3, 3, 4, 5, 6, 5, 5, 4, 6, 4, 4, 4, 6, 7, 7, 5, 3, 5, 5, 6, 8, 6, 4, 5, 6, 6]
Молоко
159
МОЛОКО
159
молоко
159
Молоко
159
МОЛОКО
159
молоко
159
Молоко
159
МОЛОКО
159
молоко
159
Молоко
159
[5, 0, 5, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 3, 5, 5, 6, 4, 5, 6, 4, 3, 4, 3, 7, 5, 5, 5, 6, 5, 3, 3, 3, 3, 5, 3, 5, 5, 4, 5, 3, 4, 5, 5, 4, 5, 4, 6, 7, 7, 4, 4, 4, 4, 5, 7, 5, 4, 5, 6, 5]
Творог
160
ТВОРОГ
160
творог
160
Творог
160
ТВОРОГ
160
творог
160
Творог
160
ТВОРОГ
160
творог
160
Творог
160
[5, 4, 5, 5, 4, 5, 4, 5, 4, 4, 4, 0, 4, 5, 3, 5, 5, 6, 4, 5, 6, 4, 3, 4, 3, 7, 5, 5, 5, 6, 5, 5, 3, 4, 3, 5, 3, 5, 5, 3, 5, 3, 4, 5, 5, 4, 5, 5, 6, 7, 7, 4, 4, 4, 5, 5, 7, 5, 4, 5, 6, 6]
Хлеб
161
ХЛЕБ
161
хлеб
161
Хлеб
161
ХЛЕБ
161
хлеб
161
Хлеб
161
ХЛЕБ
161
хлеб
161
Хлеб
161
ХЛЕБ
161
хлеб
161
Хлеб
161
ХЛЕБ
161
хлеб
161
Хлеб
161
ХЛЕБ
161
хлеб
161
Хлеб
161
ХЛЕБ
161
хлеб
161
Хлеб
161
ХЛЕБ
161
хлеб
161
Хле

Молоко
200
МОЛОКО
200
молоко
200
Молоко
200
МОЛОКО
200
молоко
200
Молоко
200
МОЛОКО
200
молоко
200
Молоко
200
[5, 0, 5, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 3, 5, 5, 6, 4, 5, 6, 4, 3, 4, 3, 7, 5, 5, 5, 6, 5, 3, 3, 3, 3, 5, 3, 5, 5, 4, 5, 3, 4, 5, 5, 4, 5, 4, 6, 7, 7, 4, 4, 4, 4, 5, 7, 5, 4, 5, 6, 5]
Творог
202
ТВОРОГ
202
творог
202
Творог
202
ТВОРОГ
202
творог
202
Творог
202
ТВОРОГ
202
творог
202
Творог
202
[5, 4, 5, 5, 4, 5, 4, 5, 4, 4, 4, 0, 4, 5, 3, 5, 5, 6, 4, 5, 6, 4, 3, 4, 3, 7, 5, 5, 5, 6, 5, 5, 3, 4, 3, 5, 3, 5, 5, 3, 5, 3, 4, 5, 5, 4, 5, 5, 6, 7, 7, 4, 4, 4, 5, 5, 7, 5, 4, 5, 6, 6]
Макароны
203
МАКАРОНЫ
203
макароны
203
Макароны
203
МАКАРОНЫ
203
макароны
203
Макароны
203
МАКАРОНЫ
203
макароны
203
Макароны
203
МАКАРОНЫ
203
макароны
203
Макароны
203
МАКАРОНЫ
203
макароны
203
Макароны
203
[5, 6, 6, 5, 4, 4, 4, 6, 5, 4, 7, 5, 5, 5, 4, 5, 5, 7, 4, 4, 7, 3, 2, 4, 6, 3, 5, 0, 4, 7, 5, 5, 3, 4, 3, 5, 3, 6, 6, 3, 6, 5, 6, 4, 6, 4, 4, 4, 6, 6, 6, 4, 4, 4, 4, 6, 7, 5, 4, 6, 7, 6]
Конфеты
205

Пакет
249
ПАКЕТ
249
пакет
249
Пакет
249
ПАКЕТ
249
пакет
249
Пакет
249
[6, 6, 7, 5, 5, 0, 4, 7, 5, 4, 6, 6, 4, 6, 5, 4, 5, 7, 5, 3, 6, 5, 5, 5, 6, 8, 5, 6, 5, 7, 6, 6, 4, 5, 4, 5, 5, 5, 5, 4, 6, 5, 6, 6, 5, 3, 5, 4, 6, 8, 7, 5, 4, 4, 5, 5, 8, 5, 4, 6, 8, 7]
Бумага
250
БУМАГА
250
бумага
250
Бумага
250
БУМАГА
250
бумага
250
Бумага
250
БУМАГА
250
бумага
250
Бумага
250
БУМАГА
250
бумага
250
Бумага
250
[4, 5, 6, 4, 4, 4, 4, 7, 4, 3, 6, 5, 4, 0, 6, 5, 5, 6, 4, 3, 7, 4, 3, 4, 6, 8, 5, 7, 3, 7, 5, 5, 3, 3, 3, 4, 2, 5, 6, 4, 6, 4, 5, 4, 6, 4, 5, 3, 6, 7, 6, 4, 3, 4, 4, 6, 9, 6, 3, 6, 6, 6]
Семена
252
СЕМЕНА
252
семена
252
Семена
252
СЕМЕНА
252
семена
252
Семена
252
СЕМЕНА
252
семена
252
Семена
252
СЕМЕНА
252
семена
252
Семена
252
[5, 5, 7, 4, 5, 4, 4, 6, 0, 3, 5, 5, 3, 4, 4, 4, 5, 4, 4, 3, 4, 4, 3, 4, 5, 7, 5, 6, 4, 7, 5, 5, 3, 3, 2, 4, 2, 5, 4, 4, 6, 5, 6, 4, 6, 4, 4, 4, 6, 7, 6, 4, 3, 4, 5, 6, 9, 6, 4, 5, 7, 5]
Шоколад
253
ШОКОЛАД
253
шоколад
253
Шоколад
253
ШОКОЛАД
253
шоколад
253
Шоколад
253

...

печенье
391
Печенье
391
ПЕЧЕНЬЕ
391
печенье
391
Печенье
391
ПЕЧЕНЬЕ
391
печенье
391
Печенье
391
[5, 5, 7, 5, 6, 4, 3, 6, 4, 3, 5, 5, 5, 5, 0, 5, 5, 6, 3, 4, 6, 4, 3, 4, 6, 8, 6, 7, 4, 7, 6, 5, 3, 3, 3, 6, 3, 5, 5, 4, 6, 4, 5, 5, 6, 4, 5, 4, 5, 7, 7, 4, 2, 4, 5, 6, 8, 6, 3, 6, 7, 6]
Чай
392
ЧАЙ
392
чай
392
Чай
392
ЧАЙ
392
чай
392
Чай
392
ЧАЙ
392
чай
392
Чай
392
ЧАЙ
392
чай
392
Чай
392
[5, 5, 6, 5, 4, 4, 4, 6, 5, 3, 6, 5, 4, 4, 5, 4, 5, 6, 4, 4, 7, 4, 3, 4, 6, 7, 5, 6, 5, 6, 5, 4, 2, 4, 0, 5, 2, 6, 6, 4, 6, 4, 5, 5, 6, 4, 5, 4, 6, 7, 6, 5, 3, 4, 4, 6, 8, 5, 2, 6, 7, 6]
Апельсины
393
АПЕЛЬСИНЫ
393
апельсины
393
Апельсины
393
[5, 5, 7, 6, 6, 4, 4, 8, 6, 4, 6, 5, 5, 5, 7, 6, 6, 7, 4, 5, 7, 4, 3, 4, 6, 6, 5, 7, 5, 8, 6, 5, 3, 4, 3, 6, 3, 6, 6, 4, 6, 5, 7, 6, 7, 5, 5, 5, 6, 0, 7, 4, 4, 4, 5, 7, 9, 7, 3, 7, 7, 6]
Бананы
394
БАНАНЫ
394
бананы
394
Бананы
394
[5, 5, 7, 5, 0, 4, 4, 6, 5, 4, 6, 5, 4, 4, 6, 6, 6, 7, 4, 5, 7, 4, 3, 4, 6, 4, 5, 4, 5, 5, 5, 5, 3, 4, 3, 4, 3, 6, 6, 4, 6, 5, 6, 6, 6, 4, 

Шоколад
435
ШОКОЛАД
435
шоколад
435
Шоколад
435
ШОКОЛАД
435
шоколад
435
Шоколад
435
ШОКОЛАД
435
шоколад
435
Шоколад
435
ШОКОЛАД
435
шоколад
435
Шоколад
435
ШОКОЛАД
435
шоколад
435
Шоколад
435
[5, 5, 7, 5, 5, 5, 4, 7, 5, 4, 0, 5, 4, 5, 5, 5, 6, 7, 4, 5, 7, 4, 2, 4, 5, 7, 5, 7, 4, 7, 6, 6, 2, 4, 3, 6, 2, 6, 6, 4, 5, 5, 5, 4, 6, 4, 5, 4, 7, 5, 7, 4, 4, 4, 4, 6, 7, 6, 4, 6, 5, 6]
Пакет
437
ПАКЕТ
437
пакет
437
Пакет
437
ПАКЕТ
437
пакет
437
Пакет
437
[6, 6, 7, 5, 5, 0, 4, 7, 5, 4, 6, 6, 4, 6, 5, 4, 5, 7, 5, 3, 6, 5, 5, 5, 6, 8, 5, 6, 5, 7, 6, 6, 4, 5, 4, 5, 5, 5, 5, 4, 6, 5, 6, 6, 5, 3, 5, 4, 6, 8, 7, 5, 4, 4, 5, 5, 8, 5, 4, 6, 8, 7]
Кукуруза
439
КУКУРУЗА
439
кукуруза
439
Кукуруза
439
КУКУРУЗА
439
кукуруза
439
Кукуруза
439
КУКУРУЗА
439
кукуруза
439
Кукуруза
439
сахар
439
Сахар
439
САХАР
439
сахар
439
Сахар
439
САХАР
439
сахар
439
Сахар
439
[5, 6, 0, 5, 4, 4, 4, 7, 5, 3, 6, 6, 0, 5, 7, 5, 6, 7, 4, 5, 7, 4, 3, 4, 7, 6, 5, 5, 5, 7, 6, 6, 3, 4, 3, 5, 3, 6, 6, 5, 6, 4, 6, 5, 5, 2, 5, 4, 7, 9, 4, 

[5, 5, 7, 5, 4, 4, 4, 7, 4, 3, 6, 5, 0, 5, 5, 5, 5, 6, 3, 3, 7, 4, 3, 4, 6, 6, 5, 5, 4, 6, 6, 5, 3, 3, 3, 5, 3, 6, 5, 4, 6, 5, 6, 4, 6, 2, 5, 4, 6, 7, 4, 4, 3, 4, 5, 6, 9, 6, 3, 6, 6, 6]
Сыр
480
СЫР
480
сыр
480
Сыр
480
СЫР
480
сыр
480
Сыр
480
СЫР
480
сыр
480
Сыр
480
[5, 5, 7, 3, 5, 4, 4, 7, 5, 4, 6, 5, 3, 5, 7, 5, 6, 7, 4, 4, 7, 4, 2, 4, 6, 8, 5, 7, 4, 7, 5, 5, 2, 3, 2, 5, 0, 6, 6, 4, 5, 5, 5, 5, 5, 4, 5, 3, 6, 6, 6, 4, 4, 3, 4, 6, 9, 6, 3, 6, 7, 6]
Пирог
481
ПИРОГ
481
пирог
481
Пирог
481
ПИРОГ
481
пирог
481
Пирог
481
[5, 5, 7, 5, 6, 4, 4, 7, 6, 4, 6, 3, 5, 5, 6, 6, 6, 8, 3, 4, 6, 4, 4, 4, 5, 8, 6, 6, 5, 7, 5, 5, 4, 4, 4, 6, 4, 7, 7, 0, 5, 5, 7, 6, 6, 5, 3, 5, 6, 8, 7, 4, 4, 5, 4, 7, 8, 6, 4, 7, 5, 6]
Хлеб
482
ХЛЕБ
482
хлеб
482
Хлеб
482
ХЛЕБ
482
хлеб
482
Хлеб
482
[5, 5, 8, 6, 6, 4, 4, 7, 5, 0, 6, 5, 5, 5, 6, 5, 6, 7, 4, 3, 8, 4, 4, 4, 7, 9, 6, 8, 5, 8, 6, 5, 4, 4, 4, 6, 4, 7, 6, 4, 7, 5, 6, 5, 7, 4, 5, 5, 7, 8, 7, 4, 2, 5, 5, 6, 9, 6, 4, 7, 7, 7]
Сосиски
483
СОСИСКИ
483
сосиски
483
Сос

Огурцы
526
ОГУРЦЫ
526
огурцы
526
Огурцы
526
ОГУРЦЫ
526
огурцы
526
Огурцы
526
[5, 5, 6, 5, 5, 4, 4, 6, 6, 4, 6, 5, 5, 5, 7, 4, 6, 6, 4, 5, 6, 4, 3, 4, 6, 7, 4, 6, 4, 7, 0, 5, 2, 4, 3, 5, 3, 6, 6, 4, 6, 5, 6, 6, 6, 5, 5, 4, 6, 8, 7, 4, 4, 3, 4, 6, 8, 7, 3, 7, 7, 5]
Пакет
531
ПАКЕТ
531
пакет
531
Пакет
531
ПАКЕТ
531
пакет
531
Пакет
531
[6, 6, 7, 5, 5, 0, 4, 7, 5, 4, 6, 6, 4, 6, 5, 4, 5, 7, 5, 3, 6, 5, 5, 5, 6, 8, 5, 6, 5, 7, 6, 6, 4, 5, 4, 5, 5, 5, 5, 4, 6, 5, 6, 6, 5, 3, 5, 4, 6, 8, 7, 5, 4, 4, 5, 5, 8, 5, 4, 6, 8, 7]
Сахар
532
САХАР
532
сахар
532
Сахар
532
САХАР
532
сахар
532
Сахар
532
САХАР
532
сахар
532
Сахар
532
[5, 5, 7, 5, 4, 4, 4, 7, 4, 3, 6, 5, 0, 5, 5, 5, 5, 6, 3, 3, 7, 4, 3, 4, 6, 6, 5, 5, 4, 6, 6, 5, 3, 3, 3, 5, 3, 6, 5, 4, 6, 5, 6, 4, 6, 2, 5, 4, 6, 7, 4, 4, 3, 4, 5, 6, 9, 6, 3, 6, 6, 6]
Сосиски
533
СОСИСКИ
533
сосиски
533
Сосиски
533
СОСИСКИ
533
сосиски
533
Сосиски
533
СОСИСКИ
533
сосиски
533
Сосиски
533
[5, 5, 7, 5, 5, 4, 4, 6, 5, 4, 6, 4, 3, 5, 7, 5, 6, 6, 4, 5, 4, 3, 3, 4,

[5, 5, 7, 5, 5, 4, 3, 6, 5, 3, 5, 4, 4, 5, 5, 6, 5, 7, 4, 4, 5, 0, 3, 4, 6, 7, 5, 7, 4, 4, 5, 5, 3, 3, 2, 4, 3, 6, 6, 4, 6, 4, 5, 5, 6, 4, 5, 4, 6, 7, 6, 4, 3, 4, 4, 6, 7, 5, 3, 6, 5, 6]
Сахар
576
САХАР
576
сахар
576
Сахар
576
САХАР
576
сахар
576
Сахар
576
САХАР
576
сахар
576
Сахар
576
САХАР
576
сахар
576
Сахар
576
САХАР
576
сахар
576
Сахар
576
[5, 4, 7, 3, 4, 4, 4, 7, 5, 4, 6, 5, 0, 5, 5, 4, 5, 7, 4, 3, 6, 4, 2, 4, 6, 6, 5, 5, 4, 6, 6, 4, 2, 4, 2, 5, 3, 5, 6, 3, 6, 5, 5, 4, 5, 2, 4, 3, 5, 6, 4, 4, 3, 3, 4, 6, 7, 6, 3, 6, 7, 5]
Пакет
579
ПАКЕТ
579
пакет
579
Пакет
579
ПАКЕТ
579
пакет
579
Пакет
579
[5, 4, 7, 5, 5, 0, 4, 7, 5, 4, 6, 6, 4, 4, 5, 4, 5, 6, 5, 3, 6, 4, 5, 4, 5, 6, 4, 5, 4, 5, 6, 5, 4, 4, 3, 5, 5, 4, 5, 4, 6, 4, 6, 6, 5, 3, 5, 3, 5, 8, 6, 4, 4, 1, 5, 5, 9, 4, 2, 6, 7, 6]
Морковь
580
МОРКОВЬ
580
морковь
580
Морковь
580
МОРКОВЬ
580
морковь
580
Морковь
580
МОРКОВЬ
580
морковь
580
Морковь
580
МОРКОВЬ
580
морковь
580
Морковь
580
[5, 4, 7, 5, 6, 4, 3, 6, 6, 4, 5, 5, 5, 5, 6, 6, 6, 7

In [290]:
arrr = [62,595,207,93,87,591,312,88,546,593,517,598,576,545,509,133,488,365,50,44,6,
        592,597,588,580,585,582,590,534,511,526,524,522,518,513,504,499,494,487,481,
        243,468,461,458,427,431,423,411,345,393,376,371,356,327,300,205,227,215,166,143,115,600]
print(products[61].arrayNames[1])
print(data[595])
centr = []
for i in arrr:
    centr.append(data[i])


сосиски
[5, 0, 5, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 3, 5, 5, 6, 4, 5, 6, 4, 3, 4, 3, 7, 5, 5, 5, 6, 5, 3, 3, 3, 3, 5, 3, 5, 5, 4, 5, 3, 4, 5, 5, 4, 5, 4, 6, 7, 7, 4, 4, 4, 4, 5, 7, 5, 4, 5, 6, 5]


In [22]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)
df.tail(15)

,ID,Name
586,A2415629-87F3-4EE9-B506-08DC596B5B40,Чиабатта 290г п/уп(Ухуд)
587,3D3FB037-C010-4EFB-B507-08DC596B5B40,ПАПА МОЖЕТ Сосиски Сочные мясн/пр 450г МГС(ОМ
588,3B94AC00-E351-4D04-B508-08DC596B5B40,YARATELLE Яйцо куриное С1 Халяль 10шт бокс(Ак
589,E7FD0D7D-6B51-4F2B-B509-08DC596B5B40,УВЕЛКА Гречка ядрица Экстра 800г(Ресурс):6
590,51B4F953-A1B8-421F-B50A-08DC596B5B40,MAKFA Макароны улитки в/с 450г (Макфа):10
591,1BDAE195-A956-4E9F-B50B-08DC596B5B40,Пакет-майка Магнит
592,5C7B3A24-83EC-4CC1-B50C-08DC596B5B40,AQUA MINERALE Питьевая вода негаз 1л пл/
593,F7A04DC5-E880-4F86-B50D-08DC596B5B40,Хлеб Домашний в/с 250г не упак(Кулинария ММ)
594,3916079E-2D72-498F-B50E-08DC596B5B40,"ОЧЕНЬ ВАЖНАЯ КОРОВА Молоко 2,5% 0,93кг п"
595,C93EBE72-233B-4EC2-B50F-08DC596B5B40,"ОЧЕНЬ ВАЖНАЯ КОРОВА Молоко 2,5% 0,93кг п"


In [167]:
class K_Means():
    
    def __init__(self, dataset, data_centroids, dist=0):
        self.dataset = dataset
        self.n_clusters = len(data_centroids)
        self.dist = dist
        self.centroids = data_centroids.copy()
        self.labels = np.array([], dtype='i')
        self.fitted = False
        self.max_n_iter = 100
        self.tolerance = .01
        
    def get_distOfEvklid(self, list1, list2):
        return np.sqrt(sum((i-j)**2 for i,j in zip(list1,list2)))
    
    def get_distOfQuadEvklid(self, list1, list2):
        return sum((i-j)**2 for i,j in zip(list1,list2))
    
    def get_distOfCity(self, list1, list2):
        return np.abs(sum((i-j) for i,j in zip(list1,list2)))
    
    def get_distOfChebyshev(self, list1, list2):
        return np.max(np.abs(sum((i-j) for i,j in zip(list1,list2))))
    
    def get_distOfDegree(self, list1, list2):
        return np.power(np.abs(sum((i-j)**3 for i,j in zip(list1,list2))),1/2)
    
    def distribute_data(self):
        self.labels = np.array([], dtype='i')
        for data in self.dataset:
            if self.dist == 0:
                self.labels = np.append(self.labels,np.array([self.get_distOfEvklid(data,center) for center in self.centroids]).argmin())
            if self.dist == 3:
                self.labels = np.append(self.labels,np.array([self.get_distOfChebyshev(data,center) for center in self.centroids]).argmin())
            if self.dist == 4:
                self.labels = np.append(self.labels,np.array([self.get_distOfDegree(data,center) for center in self.centroids]).argmin())
            if self.dist == 1:
                self.labels = np.append(self.labels,np.array([self.get_distOfQuadEvklid(data,center) for center in self.centroids]).argmin())
            if self.dist == 2:
                self.labels = np.append(self.labels,np.array([self.get_distOfCity(data,center) for center in self.centroids]).argmin())
                                                     
    def recount_centroids(self):
        for i in range(self.n_clusters):
            num = 0
            temp = np.zeros(np.shape(self.dataset)[1])
            for k,label in enumerate(self.labels):
                if label == i:
                    temp += self.dataset[k]
                    #print(temp)
                    num += 1
            if(num != 0):
                self.centroids[i] = temp/num
            
            
            
    def fit(self):
        iter = 0
        while iter < self.max_n_iter:
            prev_centroids = self.centroids.copy()
            self.distribute_data()
            self.recount_centroids()
            if self.dist == 0:
                if np.array([self.get_distOfEvklid(i,j) for i,j in zip(prev_centroids,self.centroids)]).max() < self.tolerance:
                    print(iter)
                    break
                iter += 1
            if self.dist == 1:
                if np.array([self.get_distOfQuadEvklid(i,j) for i,j in zip(prev_centroids,self.centroids)]).max() < self.tolerance:
                    print(iter)
                    break
                iter += 1
            if self.dist == 2:
                if np.array([self.get_distOfCity(i,j) for i,j in zip(prev_centroids,self.centroids)]).max() < self.tolerance:
                    print(iter)
                    break
                iter += 1
            if self.dist == 3:
                if np.array([self.get_distOfChebyshev(i,j) for i,j in zip(prev_centroids,self.centroids)]).max() < self.tolerance:
                    print(iter)
                    break
                iter += 1
            if self.dist == 4:
                if np.array([self.get_distOfDegree(i,j) for i,j in zip(prev_centroids,self.centroids)]).max() < self.tolerance:
                    print(iter)
                    break
                iter += 1
        self.fitted = True
        
    def predict(self, list2d):
        fuzzy = 3
        cut = .9
        V = np.array([], dtype='f')
        dist = np.array([[self.get_distOfEvklid(center,data) for center in self.centroids] for data in list2d])
        V = (1/dist)**fuzzy
        V = V/np.array([V.sum(axis=1)]).T
        list1d = np.array([], dtype='i')
        for i in range(len(list2d)):
            if (np.max(V, axis=1)[i] >= cut):
                for j in range(self.n_clusters):
                    if (np.max(V, axis=1)[i] == V[i,j]):
                        list1d = np.append(list1d, j+1)
            else:
                list1d = np.append(list1d, 0)
        return list1d


In [135]:
class C_Means():
    def __init__(self, dataset, data_centroids, fuzzy=2, cut=.9):
        self.dataset = dataset
        self.n_clusters = len(data_centroids)
        self.fuzzy = fuzzy
        self.cut = cut
        self.max_n_iter = 100
        self.tolerance = 1e-2
        self.fitted = False
        self.labels = np.array([], dtype='i')
        #self.centroids = self.dataset[np.random.choice(self.dataset.shape[0], size=self.n_clusters, replace=False)] + 1
        self.centroids = data_centroids.copy()
        self.U = np.array([], dtype='f')
        
    def get_dist(self, list1, list2):
        return np.sqrt(sum((i-j)**2 for i,j in zip(list1,list2)))
    
    def distribute_data(self):
        dist = np.array([[self.get_dist(center,data) for center in self.centroids] for data in self.dataset])
        #print(dist)
        self.U = (1/dist)**self.fuzzy
        self.U = self.U/np.array([self.U.sum(axis=1)]).T
        
    def recalculate_centroids(self):
        for j in range(self.n_clusters):
            self.centroids[j] = (((np.array([self.U[:,j]]).T*self.dataset)).sum(axis = 0)/(np.array([self.U[:,j]])).sum())
        
    def get_labels(self):
        self.labels = np.array([], dtype='i')
        for i in range(np.shape(self.dataset)[0]):
            if (np.max(self.U, axis=1)[i] >= self.cut):
                for j in range(self.n_clusters):
                    if (np.max(self.U, axis=1)[i] == self.U[i,j]):
                        self.labels = np.append(self.labels, j+1)
            else:
                self.labels = np.append(self.labels, 0)
            
    def fit(self):
        iter = 0
        while iter < self.max_n_iter:
            prev_centroids = self.centroids.copy()
            self.distribute_data()
            self.get_labels()
            self.recalculate_centroids()
            if np.array([self.get_dist(i,j) for i,j in zip(prev_centroids,self.centroids)]).max()\
            < self.tolerance:
                print(iter)
                break
            iter += 1
        self.fitted = True
        
    def predict(self, list2d):
        V = np.array([], dtype='f')
        dist = np.array([[self.get_dist(center,data) for center in self.centroids] for data in list2d])
        V = (1/dist)**self.fuzzy
        V = V/np.array([V.sum(axis=1)]).T
        list1d = np.array([], dtype='i')
        for i in range(len(list2d)):
            if (np.max(V, axis=1)[i] >= self.cut):
                for j in range(self.n_clusters):
                    if (np.max(V, axis=1)[i] == V[i,j]):
                        list1d = np.append(list1d, j+1)
            else:
                list1d = np.append(list1d, 0)
        return list1d

In [150]:
print(products[53].arrayNames[1])
#print(data)
print(np.zeros(np.shape(data)[1]))

маска
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [201]:
def get_dist(list1, list2):
    return np.sqrt(sum((i-j)**2 for i,j in zip(list1,list2)))

In [291]:
kmeans = K_Means(data, centr, dist=0)
print(kmeans.labels)
print("Первые центры")
print(kmeans.centroids)

kmeans.distribute_data()
print(kmeans.labels)

kmeans.recount_centroids()
print("Вторые центры")
print(kmeans.centroids)

kmeans.fit()

[]
Первые центры
[[0, 5, 6, 6, 6, 6, 6, 6, 5, 5, 6, 6, 5, 4, 7, 6, 6, 6, 6, 5, 6, 5, 6, 6, 6, 8, 5, 8, 5, 7, 6, 5, 5, 5, 6, 6, 5, 6, 6, 5, 7, 4, 5, 5, 5, 5, 6, 5, 6, 8, 6, 4, 5, 5, 6, 7, 8, 7, 4, 6, 6, 4], [5, 0, 5, 5, 4, 4, 4, 5, 4, 4, 4, 4, 4, 5, 3, 5, 5, 6, 4, 5, 6, 4, 3, 4, 3, 7, 5, 5, 5, 6, 5, 3, 3, 3, 3, 5, 3, 5, 5, 4, 5, 3, 4, 5, 5, 4, 5, 4, 6, 7, 7, 4, 4, 4, 4, 5, 7, 5, 4, 5, 6, 5], [5, 6, 0, 5, 4, 4, 4, 7, 5, 3, 6, 6, 0, 5, 7, 5, 6, 7, 4, 5, 7, 4, 3, 4, 7, 6, 5, 5, 5, 7, 6, 6, 3, 4, 3, 5, 3, 6, 6, 5, 6, 4, 6, 5, 5, 2, 5, 4, 7, 9, 4, 5, 4, 4, 5, 7, 9, 6, 4, 6, 7, 6], [6, 5, 6, 0, 5, 5, 4, 6, 5, 4, 5, 5, 5, 5, 6, 5, 5, 6, 4, 5, 7, 3, 2, 4, 5, 7, 6, 6, 4, 6, 6, 5, 3, 4, 3, 5, 3, 6, 6, 5, 6, 3, 4, 5, 5, 5, 4, 3, 6, 7, 7, 4, 4, 4, 4, 6, 8, 5, 4, 5, 7, 6], [5, 5, 7, 5, 0, 4, 4, 6, 5, 4, 6, 5, 4, 4, 6, 6, 6, 7, 4, 5, 7, 4, 3, 4, 6, 4, 5, 4, 5, 5, 5, 5, 3, 4, 3, 4, 3, 6, 6, 4, 6, 5, 6, 6, 6, 4, 5, 5, 6, 7, 7, 4, 4, 4, 5, 5, 9, 5, 3, 5, 7, 6], [5, 4, 7, 5, 5, 0, 4, 7, 5, 4, 6, 6, 4, 4,

Вторые центры
[array([0.71052632, 5.        , 6.71052632, 6.        , 6.        ,
       5.31578947, 5.31578947, 6.71052632, 5.02631579, 5.        ,
       6.71052632, 5.31578947, 4.31578947, 4.71052632, 6.97368421,
       6.        , 6.        , 6.71052632, 5.31578947, 5.        ,
       6.71052632, 4.34210526, 5.31578947, 5.31578947, 6.71052632,
       7.94736842, 5.        , 7.97368421, 4.28947368, 7.        ,
       6.        , 4.28947368, 4.31578947, 5.        , 5.31578947,
       6.        , 4.31578947, 6.        , 6.71052632, 5.        ,
       6.28947368, 4.71052632, 5.71052632, 5.        , 5.71052632,
       4.31578947, 5.31578947, 5.        , 5.28947368, 7.28947368,
       6.02631579, 4.71052632, 5.        , 5.02631579, 5.31578947,
       7.        , 7.97368421, 7.        , 4.71052632, 6.        ,
       6.71052632, 4.71052632]), array([4.86956522, 0.26086957, 5.95652174, 4.82608696, 4.34782609,
       3.91304348, 4.        , 5.56521739, 4.43478261, 3.86956522,
       4.30434

5


In [308]:
print(products[18].arrayNames[1])
#print(kmeans.centroids[20])
#print(kmeans.centroids[21])
#print(data[0])
#print(get_dist(kmeans.centroids[20], data[0]))
#print(get_dist(kmeans.centroids[21], data[0]))

кино


In [307]:
print(np.shape(kmeans.centroids))
kmeans2 = K_Means(data, kmeans.centroids, dist=0)
print(kmeans2.labels)
print("Первые центры")
print(kmeans2.centroids)

kmeans2.distribute_data()
print(kmeans2.labels)

(62, 62)
[]
Первые центры
[array([0.71052632, 5.        , 6.71052632, 6.        , 6.        ,
       5.31578947, 5.31578947, 6.71052632, 5.02631579, 5.        ,
       6.71052632, 5.31578947, 4.31578947, 4.71052632, 6.97368421,
       6.        , 6.        , 6.71052632, 5.31578947, 5.        ,
       6.71052632, 4.34210526, 5.31578947, 5.31578947, 6.71052632,
       7.94736842, 5.        , 7.97368421, 4.28947368, 7.        ,
       6.        , 4.28947368, 4.31578947, 5.        , 5.31578947,
       6.        , 4.31578947, 6.        , 6.71052632, 5.        ,
       6.28947368, 4.71052632, 5.71052632, 5.        , 5.71052632,
       4.31578947, 5.31578947, 5.        , 5.28947368, 7.28947368,
       6.02631579, 4.71052632, 5.        , 5.02631579, 5.31578947,
       7.        , 7.97368421, 7.        , 4.71052632, 6.        ,
       6.71052632, 4.71052632]), array([4.85714286, 0.        , 5.85714286, 4.80952381, 4.33333333,
       3.85714286, 4.        , 5.47619048, 4.52380952, 3.85714286,
  

[20 20 20 20 20 20 20 55 55 19 13 13 13 19 19  0  0 55 55 55 55  0  0  0
 42  0 22  0  0  0 22 18 52  0 22 22  0 56 19 19 19 52 22  0 19 56  0 22
  0 18 18 18 22  0 22 22  0  0  0 46 22 52  0 52  0  0  0  0 22 58 42  0
 58  0  0 42 42  0  0 13 59 18 24 61 17 16  5  4  7 19  0 42 22  3  2  8
 18 45 59 59  5 36  5 40  8 10 18 18 18  2  6  8 59 59  5 60 10 58 18 13
 12 34  5 52 34  8 17 18 18  5  9  1 14 15 42 59 18 18 13 28 59 59  8 59
 12  5  6 59 61  2 18 52 58 52 26 32 52 34  8  1 11 36  4  9  1 36 58  8
 61  2 11  4  5 23 10 40 59 59 61  9  2 12  5 12  8  5  1 29 11 55 55  4
 18 59 42 22  5 29  4 48  1 48 11 27 59 19 21  2 18 42 12 61 34 36 10 57
 36  6  5  9  5  4 61 22 27  8 11 55 11 59 26 26 42 10 59  3 45 34 46 59
 59  9 59 40 34 12 18  2 42  5 13 42  8 10 59 18 16  0 45  9 24 36 34 34
 46 23  4  5  9  1 11  0 42  0 33  2 61  6 59 10  9 45  5 22 42  0 24  5
 49 18  6 10 10 34 52 59 35 58 34  8 54 18 59 34 12  2 58  9 34 59 45 61
 59  3 34 46 33 34 59 55 33 53 53 53 53 53 53 53 21

In [83]:
np.seterr(divide='ignore', invalid='ignore')
cmeans = C_Means(data, centroidsProduct, fuzzy =3, cut=.6)
cmeans.fit()

[[13.60147051 13.82027496 22.5166605  ... 17.91647287 21.21320344
  15.09966887]
 [13.60147051 13.82027496 22.5166605  ... 17.91647287 21.21320344
  15.09966887]
 [13.60147051 13.82027496 22.5166605  ... 17.91647287 21.21320344
  15.09966887]
 ...
 [14.35270009 12.9614814  23.19482701 ... 18.70828693 22.11334439
  18.08314132]
 [14.         13.11487705 22.49444376 ... 16.43167673 20.80865205
  17.17556404]
 [14.59451952 12.4498996  24.63736999 ... 19.5192213  23.23790008
  14.49137675]]
[[ 9.78910639  8.85888995 23.44010442 ... 14.02512439 20.64370342
  11.9011556 ]
 [ 9.78910639  8.85888995 23.44010442 ... 14.02512439 20.64370342
  11.9011556 ]
 [ 9.78910639  8.85888995 23.44010442 ... 14.02512439 20.64370342
  11.9011556 ]
 ...
 [11.02667671  9.26639313 25.82564168 ... 15.99088237 22.95233402
  13.73066343]
 [ 9.99660035  8.2540912  23.40870364 ... 13.73708643 20.55935112
  11.79117398]
 [10.35231907  9.05540034 25.2165747  ... 15.44422509 22.3467439
  13.10020452]]
[[nan nan nan ...

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

In [84]:
print(cmeans.centroids)

[array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, 

In [47]:
kmeans = K_Means(data, centroidsProduct, dist=0)
print(kmeans.labels)
#print(kmeans.centroids)

kmeans.distribute_data()
print(kmeans.labels)
print(len(kmeans.labels))

[]
[21 21 21 21 21 21 21 17 17 19 32 32 32 19 19  0  0 49 17 17 49  0  0  0
 32  0 32  0  0  0 32 32 23  0 32 32  0 56 19 19 19 23 32  0 19 56  0 32
  0 32 32 32 32  0 32 32  0  0  0 46 36 34  0 34  0  0  0  0 32 18 22  0
 18  0  0  9 22  0  0 22  6 53 32 32 32 16  5  4 45  1  0  9 32 32 12 32
 32 32 32 32  5 32 53 22 32 32 32 32 32 12  6 32 32 32  5 18 21 58 32 58
 12 52  5 34 18 32 32 32 32  5  9 32 34  6 22 32 32 32 32 47 32 18 32 32
 12  5  6 32 32 12 32 34 33 34 26 32 34 34  8 32 21 22  4  9 32 36 58  8
 32 41 21  4  5 32 32 22 32 32 32  9 41 12  5 32  8  5 32 32 21 17 17 34
 32  6  6 32  5 32  4 32 32 32 21 18 32 55 21 12 32  6 12 32 32 36 32 18
 36  6  5  9  5  4 32 22 22  8 21 56 21 32 32 18 32 32 22 32 45 34 46 32
  6  9 36 22 32 12 32 41  6  5 58 18 32 21 32 32 16  0 45  9 32 32 18 34
 46 32  4  5  9 32 21  0 22  0 47 36 32  6 32 32  9 21  5 32 22  0 32 53
 47 32  6 32 32 18 34 32 18 34 21 32 22 32 32 32 12 41 53  9 18 22 45 32
  6 32 34 46  5 18 32 56  5 53 53 53 53 53 53 53

In [86]:
from sklearn.cluster import KMeans

In [93]:
print(products[7].arrayNames[1])

консервы


In [96]:
kmeans = KMeans(n_clusters=62, random_state=0).fit(data)
print(kmeans.labels_)

[17 17 17 17 17 17 17  7  7  0  6  6  6  0  0  3  3 11  7  7 11  3  3  3
  6  3  5  3  3  3  5 27 30  3  5  5  3  1 58 58 58 30  5  3 58  1  3  5
  3 27 27 27  5  3  5  5  3  3  3 45  3 30 61 30 61  3  3  3  5 52  6 61
 52 61 61 36  6 61  3 56 13 27 23 29 24 37 10 16  2 33 61 36  5 35 31 24
  2 43 21 21 14 18 10 54 24 15  2  2  2 31 20 24 21 21 14 13 60 27  2 56
 44 41 14 18 18 24 27 27  2 14 22  9 51  6 18 21  2 27  6 27 46 21 24 13
 44 14 20 21 46 31  2 18 52 30 42 21 50 50 34  9 26 41 16 41  9 18  8 34
 29 19 26 16 10 47 15 54 21 21 29 22 32 44 14 25 34 10  9 21 26  7  7 16
  2 21 49  5 10 21 16 52  9 46 26 12 46 57  4 31  2 49 44 46 47 18 15 27
 13 20 14 41 10 16 29  6 12 34 26 48 26 18 59 42 36 15 13 35 43 50 39 13
 13 41 13 54 47 44  2 32 49 14 56 18 24 60 13  2 37  3 43 41 13 13  6 50
 39 47 16 10 22  9 26  3 45 61 27 18 29 20  6 15 22 43 14  5 45 61 23 10
 53  2 20 15 15 21 30 13  6 18  6 24  6  2 13 18 44 32 18 22  6 13 43 29
 13 35 50 39  2 18 13 28  2 40 40 40 40 40 40 40  4

# YandexGPT

In [ ]:
# Идентификатор ключа: ajemrbpv2o768338jea6
# Ваш секретный ключ: AQVN1trz8WDmQMaJuUbsMmnT1f4fk4yKxbKeGAt8
# Идентификатор каталога: b1gs6mhou6vpmtqakh4k
# Ваш секретный ключ: AQVNzMtBHIVdndOXp5-fqNVDSYhyTy_4iwpVpV6v
#Идентификатор ключа: ajecpd0hp1atjn9pf9sv

In [3]:
kindProduct = []
print(kindProduct)
#print(result)

[]


In [35]:
print(len(kindProduct))

434


In [7]:
import requests
import json
import time
#i = 0
#kindProduct = []
for i in range(269, len(df.Name)):
    prompt = {
        "modelUri": "gpt://b1gs6mhou6vpmtqakh4k/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.6,
            "maxTokens": "2000"
        },
        "messages": [
                {
                    "role": "system",
                    "text": "Одним словом определи товар и его категорию через пробел. Без уточнений. Что-то типа: молоко - продукт, кукуруза - продукт, фасоль - продукт, бананы - продукт, пакет - тара, подписка - услуга"
                },
            {
                "role": "user",
                "text": f'Одним словом определи продукт и его категорию через дефис для "{df.Name[i]}"'
            }
        ]
    }
    
    
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNzMtBHIVdndOXp5-fqNVDSYhyTy_4iwpVpV6v"
    }
    time.sleep(1)
    response = requests.post(url, headers=headers, json=prompt)
    result = response.text
    print(i)
    print(df.Name[i])
    res = json.loads(result)
    alt = res["result"]["alternatives"]
    for a in alt:
        kindProduct.append(a["message"]["text"])
        #print(nameProduct)
        print(a["message"]["text"])

269
Пакет-майка Магнит
Пакет — тара.
270
AQUA MINERALE Питьевая вода негаз 1л пл/
Вода — продукт.
271
Хлеб Домашний в/с 250г не упак(Кулинария ММ)
Хлеб — продукт.
272
ОЧЕНЬ ВАЖНАЯ КОРОВА Молоко 2,5% 0,93кг п
Молоко — продукт.
273
ОЧЕНЬ ВАЖНАЯ КОРОВА Молоко 2,5% 0,93кг п
Молоко — продукт.
274
6 СОТОК Кукуруза сладкая в/с 340г ж/б:12
Кукуруза — продукт.
275
НАЦИОНАЛЬ Рис для плова 1с 900г м/уп(Ангстрем
Рис — продукт.
276
ОЧЕНЬ ВАЖНАЯ КОРОВА Творог 5% 400г ст/бэ
Творог — продукт.
277
БАНАНЫ 1кг
Фрукты.
278
DELIKAISER Сосиски Молочные ГОСТ 0,32кг МГС (
Сосиски – продукт.


In [51]:
for i in kindProduct:
    print(i)
    print("==========================================================")

Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Услуга.
Шпроты.
Морковь по-корейски
Сосиски
Сгущёнка.
Кетчуп
Товар — упаковка.
Бананы
Консервы.
Магнит
Услуга.
Устройство.
Услуга.
К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?
Кукуруза
Семена
Молоко
Зубная паста.
Шоколад
Жевательная резинка.
Пакет
Хлебец.
Пакет
Цикорий
Семена
Шоколад
Молоко
Молоко
Молоко
Кукуруза
Творог
Семена
Сырок.
С

In [48]:
for i in kindProduct:
    print(i)
    print("==========================================================")

Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Продукт.

Я не могу определить продукт, который вы ищете, поскольку в запросе не хватает информации.  

Вот варианты товаров, которые можно назвать шпротами:

1. Шпроты «За Родину» из балтий

In [36]:
for i in kindProduct:
    print(i)
    print("==========================================================")

Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Подписка.
Устройство.

SSD накопители обычно не относят к определённым категориям продуктов, поскольку они являются устройствами компьютерной памяти. Характеристики, такие как объём памяти, скорость чтения и записи, интерфейс подключения и форм-фактор, могут варьироваться в зависимости от модели SSD накопителя.
Услуга.

Остальные товары — это продукты питания, которые можно разделить на группы:
— 

In [31]:
print(len(kindProduct))
print(kindProduct)

60
['Подписка.', 'Подписка.', 'Подпиской.\n\nЯ могу предположить, что вы имели в виду не вид продукта, а маркетинговый термин «подписка», который означает форму договора на предоставление услуг, доступного по требованию клиента через регулярные промежутки времени за регулярно вносимую плату.', 'Подписка.', 'Подпиской.\n\nСлово «продукт» в этом контексте может быть интерпретировано как подписка на сервис «КиноПоиск».', 'Подпиской.\n\nЕсли вы хотели узнать родовое название этого понятия, то вот оно.', 'Подпиской.\n\nВозможны и другие варианты в зависимости от контекста.', 'Чеквейер.', 'Чеквейор. Вероятно, имелся в виду термин «чеквейер». Чеквейеры используются для проверки массы товара, его размера и других параметров, а также для автоматической сортировки продукции.', 'Билет.', 'Сбор.', 'Такс-фри. Вероятно, речь идёт о сервисном сборе за оформление возврата налога на добавленную стоимость (НДС), который может быть включён в стоимость билетов.', 'Такс-фри. Это налог, который можно вернут

In [26]:
print(kindProduct)

['Подпиской.', 'Подпиской.', 'Подпиской.\n\nВозможно, имелся в виду **абонемент** — договор, по которому одна сторона обязуется выполнить в пользу другой стороны определённые услуги. В данном случае покупатель приобретает право пользоваться определённым набором услуг в течение определённого периода времени. Обычно термин «абонемент» используется в отношении билетов на посещение культурных мероприятий (например, театров, музеев и т. п.), а также в отношении других товаров и услуг, доступ к которым ограничен.', 'Подписaние.', 'Подписка.', 'Подписка.', 'Подпиской.\n\nСлово «товар» в этом контексте используется в обобщённом значении — как всё, что продаётся и покупается.', 'Скорей всего, вы имеете в виду **анкету** для проверки платёжеспособности.\n\nЭто стандартный документ, который используют кредиторы для предварительной оценки кредитоспособности потенциального заёмщика и принятия решения о выдаче кредита. В анкету обычно включают вопросы о доходах, образовании, семейном положении, нали

In [ ]:
df[product.arrayNames[1]] = kindProduct
df

In [17]:
import json

res = json.loads(result)
alt = res["result"]["alternatives"]
for a in alt:
    print(a["message"]["text"])

Продукт.


In [3]:
kindProduct = []
for item in df.YandexGPT:
    kindProduct.append(item.replace('.',''))
print(kindProduct)

['Подписка', 'Подписка', 'Подписка', 'Подписка', 'Подписка', 'Подписка', 'Подписка', 'Услуга', 'Услуга', 'Билет', 'Сбор', 'Сбор', 'Сбор', 'Билет', 'Билет', 'Подписка', 'Подписка', 'Без уточнения можно предположить, что «51139414» — это **консервы**', 'Услуга', 'Услуга', 'Без уточнения можно предположить, что «51139414» — это **подписка**', 'Подписка', 'Подписка', 'Подписка', 'Полис', 'Подписка', 'Подписка', 'Подписка', 'Подписка', 'Подписка', 'Подписка', 'Билет', 'Контент', 'Подписка', 'Подписка', 'Подписка', 'Подписка', 'Продукт – **конфеты**', 'Билеты', 'Билеты – это **транспорт**', 'Билеты', 'Контент', 'Подписка', 'Подписка', 'Билеты – это **транспорт**', 'Продукт — «конфеты»', 'Подписка', 'Подписка', 'Подписка', 'Билет', 'Билет', 'Билет', 'Подписка', 'Подписка', 'Подписка', 'Счёт', 'Подписка', 'Подписка', 'Подписка', 'Услуга', 'Услуга', 'Накопитель', 'Услуга', 'Устройство', 'Услуга', '**Связь**', 'Подписка', 'Подписка', 'Подписка', 'Швабра', 'Скатерть', 'Услуга', 'Швабра', 'Услуга'

In [23]:
product = []
category = []
for item in df.YandexGPT:
    tmp = item.split("-")
    product.append(tmp[0].strip("*").strip('.').strip())
    if(len(tmp) > 1):
        category.append(tmp[1].strip("*").strip('.').strip())
    else:
        category.append("Неизвестно")
print(len(product))
print(product)
print(len(category))
print(category)

121
['Игра', 'Услуга', 'Текстиль', 'Растворимый кофе', 'Фрукты', 'Консервы', 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?', 'Лекарство', 'Лекарство', 'Кофе', 'Хлеб', 'Сок', 'Фрукты', 'Фрукт', 'Туалетная бумага, гигиена', 'Мясной продукт', 'Фрукты', 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?', 'Снек', 'Йогурт', 'Йогурт', 'Блюдо', 'Блюдо', 'Блюдо', 'Хлеб', 'Салат', 'Блюдо', 'Мясопродукты', 'Фрукты', 'Напиток', 'Сладость', 'Сладость', 'Лекарство', 'Лекарство', 'Зернопродукт', 'Томат', 'Мясопродукты', 'Выпечка', 'Растворимый кофе', 'Фастфуд', 'Игрушка', 'Сахар', 'Сок', 'Хлебобулочное изделие', 'Бананы', 'Йогурт', 'Шоколад', 'Полуфабрикат', 'Консервы', 'Блюдо', 'Шоколад', 'Фрукт', 'Мыло', 'Мясное изделие', 'Молоко сгущённое', 'Фрукты', 'Сыр', 'Жевательная резинка', 'Зубная паста', 'Йогурт', 'Йогурт', 'Сок', 'Мандарины', 'Натс', 'Злак', 'Бананы', 'Фрукты', 'Фрукты', 'Фрукт', 'Фрукты', 'БАД', 'Мороженое', 'Морс', 'Вода', 'Процедура', 'Морожено

In [27]:
#print(kindProduct)
SERVER = '(local)'
DATABASE = 'Checks'
USERNAME = 'VORONVI\Vladi'

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};Trusted_Connection=Yes;TrustServerCertificate=Yes'
conn = pyodbc.connect(connectionString)

sql = "UPDATE ItemData SET YandexGPT = ? WHERE Id = ?"
val = ("Valley 345", "Canyon 123")
cursor = conn.cursor()

SQL_QUERY = """
SELECT 
 Id, 
name
FROM 
ItemData;
"""

for i in range(121):
    val = (product[i], df.ID[i])
    cursor.execute(sql, val)    


In [28]:
conn.commit()

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b_lora"

# Загружаем модель
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
	config.base_model_name_or_path,
	torch_dtype=torch.float16,
	device_map="auto"
)
model = PeftModel.from_pretrained(
	model,
	MODEL_NAME,
	torch_dtype=torch.float16
)
model.eval()

# Определяем токенайзер
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Функция для обработки запросов
def generate(model, tokenizer, prompt, generation_config):
	data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
	data = {k: v.to(model.device) for k, v in data.items()}
	output_ids = model.generate(
    	**data,
    	generation_config=generation_config
	)[0]
	output_ids = output_ids[len(data["input_ids"][0]):]
	output = tokenizer.decode(output_ids, skip_special_tokens=True)
	return output.strip()

# Формируем запрос
PROMT_TEMPLATE = '<s>system\nТы — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s><s>user\n{inp}</s><s>bot\n'
inp = 'Какое расстояние до Луны?'
prompt = PROMT_TEMPLATE.format(inp=inp)

# Отправляем запрос в llm
output = generate(model, tokenizer, prompt, generation_config)

print(output)

adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

C:\Users\Vladi\anaconda3\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vladi\.cache\huggingface\hub\models--IlyaGusev--saiga_mistral_7b_lora. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

C:\Users\Vladi\anaconda3\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vladi\.cache\huggingface\hub\models--Open-Orca--Mistral-7B-OpenOrca. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.

# DataSet

In [77]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset, Value, ClassLabel, Features

In [78]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

labels = df["YandexGPT"].unique().tolist()
print(labels)

ClassLabels = ClassLabel(num_classes=len(labels), names=labels)


C:\Users\Vladi\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['Кукуруза', 'Йогурт', 'Яйца', 'Крупа', 'Игра', 'Пакет', 'Вода', 'Молоко', 'Семена', 'Творог', 'Цикорий', 'Шоколад', 'Услуга', 'Билеты', 'Сбор', 'Подписка', 'Неизвестно', 'Полис', 'Билет', 'Контент', 'Тариф', 'Жилье', 'Платёж', 'Устройство', 'Доставка', 'Швабра', 'Скатерть', 'Флешка', 'Текстиль', 'Бумага', 'Растворимый кофе', 'Шпроты', 'Морковь по-корейски', 'Сосиски', 'Сгущенка', 'Кетчуп', 'Бананы', 'Консервы', 'Артефакт', 'Фасоль', 'Зубная паста', 'Леденец', 'Хлебец', 'Сырок', 'Виноград', 'Батончик', 'Сахар', 'Рулет', 'Лекарство', 'Сгущёнка', 'Хлеб', 'Печенье', 'Десерт', 'Кофе', 'Килька', 'Масло', 'Концентрат', 'Шампунь', 'Сок', 'Нектар', 'Чай', 'Сыр', 'Мука', 'Семечки', 'Банан', 'Мороженое', 'Конфеты', 'Туалетная бумага', 'Макароны', 'Мясной продукт', 'Лак', 'Майонез', 'Рис', 'Копия', 'Сдоба', 'Слойка', 'Круассан', 'Блюдо', 'Лобио', 'Салат', 'Лимоны', 'Соус', 'Пита', 'Мясопродукты', 'Наушники', 'Напиток', 'Выпечка', 'Торт', 'Морковь', 'Апельсины', 'Товар', 'Чипсы', 'Скретч-кар', 'На

In [166]:
trainLabels = []
testLabels = []
for label in train.YandexGPT:
    trainLabels.append(ClassLabels.str2int(label))
for label in test.YandexGPT:
    testLabels.append(ClassLabels.str2int(label))

In [167]:
trainDf = pd.DataFrame({"text": train.Name, "label": trainLabels})
testDf = pd.DataFrame({"text": test.Name, "label": testLabels})
features = Features({"text": Value("string"), "label": ClassLabel(num_classes=len(labels), names=labels)})
d = {'train':Dataset.from_dict({'label':trainLabels,'text':train.Name}, features=features),
     'validation':Dataset.from_dict({'label':testLabels,'text':test.Name}, features=features),
     }
#d = {'train': Dataset.from_pandas(train, features=features),
#     'validation': Dataset.from_pandas(test, features=features),
#}

raw_datasets = DatasetDict(d)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 480
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 121
    })
})

In [168]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'label': 35, 'text': 'РОС-Щ.Д.Шок.РОССИЙСК.темный 82г'}

In [169]:
raw_train_dataset.features

{'label': ClassLabel(names=['Подписка', 'Услуга', 'Билет', 'Сбор', 'Без уточнения можно предположить, что «51139414» — это **консервы**', 'Без уточнения можно предположить, что «51139414» — это **подписка**', 'Полис', 'Контент', 'Продукт – **конфеты**', 'Билеты', 'Билеты – это **транспорт**', 'Продукт — «конфеты»', 'Счёт', 'Накопитель', 'Устройство', '**Связь**', 'Швабра', 'Скатерть', 'Флешка', 'Бумага', 'Кофе', 'Шпроты', 'Морковь', 'Сосиски', 'Сгущёнка', 'Кетчуп', 'Пакет', 'Бананы', 'Килька', 'Магнит', 'К сожалению, я не могу ничего сказать об этом Давайте сменим тему?', 'Кукуруза', 'Семена', 'Молоко', 'Зубная паста', 'Шоколад', 'Жевательная резинка', 'Хлебец', 'Цикорий', 'Творог', 'Сырок', 'Виноград', 'Натс', 'Сахар', 'Рулет', 'Лекарство', 'Упаковка', 'Хлеб', 'Печенье', 'Десерт', 'Консервы', 'Масло', 'Конфитюр', 'Шампунь', 'Сок', 'Нектар', 'Йогурт', 'Чай', 'Фрукты', 'Сыр', 'Мука', 'Яйцо', 'Батончик', 'Вафельный стаканчик', 'Конфета', 'Семечки', 'Туалетная бумага', 'Макароны', 'Колбас

# bert-base-multilingual-cased

In [170]:
from transformers import AutoTokenizer

checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_text = tokenizer(raw_datasets["train"]["text"])

In [171]:
inputs = tokenizer("ЧУДО Йогурт пит дыня манго персик 1,9% 2", "Йогурт")
inputs

{'input_ids': [101, 532, 16859, 22681, 18002, 518, 67456, 20681, 556, 15811, 545, 12119, 10385, 97744, 75189, 61381, 13700, 10510, 122, 117, 130, 110, 123, 102, 518, 67456, 20681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [172]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["text"],
    padding=True,
    truncation=True,
)

In [173]:
def tokenize_function(example):
    return tokenizer( example["text"], truncation=True)

In [174]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 480
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 121
    })
})

In [175]:
tokenized_datasets["train"].features

{'label': ClassLabel(names=['Подписка', 'Услуга', 'Билет', 'Сбор', 'Без уточнения можно предположить, что «51139414» — это **консервы**', 'Без уточнения можно предположить, что «51139414» — это **подписка**', 'Полис', 'Контент', 'Продукт – **конфеты**', 'Билеты', 'Билеты – это **транспорт**', 'Продукт — «конфеты»', 'Счёт', 'Накопитель', 'Устройство', '**Связь**', 'Швабра', 'Скатерть', 'Флешка', 'Бумага', 'Кофе', 'Шпроты', 'Морковь', 'Сосиски', 'Сгущёнка', 'Кетчуп', 'Пакет', 'Бананы', 'Килька', 'Магнит', 'К сожалению, я не могу ничего сказать об этом Давайте сменим тему?', 'Кукуруза', 'Семена', 'Молоко', 'Зубная паста', 'Шоколад', 'Жевательная резинка', 'Хлебец', 'Цикорий', 'Творог', 'Сырок', 'Виноград', 'Натс', 'Сахар', 'Рулет', 'Лекарство', 'Упаковка', 'Хлеб', 'Печенье', 'Десерт', 'Консервы', 'Масло', 'Конфитюр', 'Шампунь', 'Сок', 'Нектар', 'Йогурт', 'Чай', 'Фрукты', 'Сыр', 'Мука', 'Яйцо', 'Батончик', 'Вафельный стаканчик', 'Конфета', 'Семечки', 'Туалетная бумага', 'Макароны', 'Колбас

In [176]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [177]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "text"]}
[len(x) for x in samples["input_ids"]]

[25, 25, 18, 25, 29, 10, 20, 16]

In [178]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 29]),
 'token_type_ids': torch.Size([8, 29]),
 'attention_mask': torch.Size([8, 29]),
 'labels': torch.Size([8])}

In [179]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [183]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [184]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [185]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=180, training_loss=3.5049034966362846, metrics={'train_runtime': 479.0343, 'train_samples_per_second': 3.006, 'train_steps_per_second': 0.376, 'total_flos': 22057550882016.0, 'train_loss': 3.5049034966362846, 'epoch': 3.0})

# Валидация

In [186]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(121, 134) (121,)


In [189]:
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
def compute_metrics(eval_preds):
    #metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [192]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [193]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [194]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 28]),
 'token_type_ids': torch.Size([8, 28]),
 'attention_mask': torch.Size([8, 28])}

In [195]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [196]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(4.9119, grad_fn=<NllLossBackward0>) torch.Size([8, 134])


In [197]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\Vladi\anaconda3\lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [198]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

180


In [200]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

In [201]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/180 [00:00<?, ?it/s]

In [204]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

C:\Users\Vladi\anaconda3\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [217]:
pipe("БАНАНЫ 1кг")

[[{'label': 'LABEL_0', 'score': 0.027497107163071632},
  {'label': 'LABEL_1', 'score': 0.012643837369978428},
  {'label': 'LABEL_2', 'score': 0.005668966565281153},
  {'label': 'LABEL_3', 'score': 0.00819145143032074},
  {'label': 'LABEL_4', 'score': 0.0031844221521168947},
  {'label': 'LABEL_5', 'score': 0.0037963194772601128},
  {'label': 'LABEL_6', 'score': 0.005351901985704899},
  {'label': 'LABEL_7', 'score': 0.004261556547135115},
  {'label': 'LABEL_8', 'score': 0.004481500945985317},
  {'label': 'LABEL_9', 'score': 0.010676776058971882},
  {'label': 'LABEL_10', 'score': 0.004692866001278162},
  {'label': 'LABEL_11', 'score': 0.0031394422985613346},
  {'label': 'LABEL_12', 'score': 0.007478203158825636},
  {'label': 'LABEL_13', 'score': 0.006057640537619591},
  {'label': 'LABEL_14', 'score': 0.003672346705570817},
  {'label': 'LABEL_15', 'score': 0.004918643739074469},
  {'label': 'LABEL_16', 'score': 0.007951938547194004},
  {'label': 'LABEL_17', 'score': 0.0030836444348096848},

In [218]:
ClassLabels.int2str(58)

'Фрукты'